In [5]:
# Required packages:
import elasticsearch
import math
import numpy as np
import os
import pytest
import random
import requests
import tarfile

from collections import Counter
from collections import defaultdict
from elasticsearch import Elasticsearch

import re
from elasticsearch import Elasticsearch
from elasticsearch.helpers import parallel_bulk,scan
from json import JSONDecodeError
from collections import defaultdict
import json


from parse_dbpedia import *

# YOUR CODE HERE
import pprint as pprint
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
import pandas as pd
import collections as cl
import string

[nltk_data] Error loading stopwords: <urlopen error [Errno -3]
[nltk_data]     Temporary failure in name resolution>


In [7]:
# load queries

from smart_dataset.evaluation.dbpedia.evaluate import load_type_hierarchy, evaluate, get_type_path
with open('./data/train_set', 'r') as f:
    train_set = json.load(f)
    
type_hierarchy, max_depth = load_type_hierarchy('./smart_dataset/evaluation/dbpedia/dbpedia_types.tsv')


Loading type hierarchy from ./smart_dataset/evaluation/dbpedia/dbpedia_types.tsv... 760 types loaded (max depth: 7)


In [8]:
#save features as a dictionary format in file

def save_features(features, filename):
    with open(filename,'w') as f:
        json.dump(features,f)
    f.close()
    return

#load features as dictionary
def load_features(filename):
    try:
        with open(filename,'r') as f:
            features = json.load(f)
        return features 
    except:
        return None
    

In [13]:
#TC_index = load_features('./data/document_TC_short.json')
#TC_index = load_features('./data/document_TC_all.json')

#with open('./data/document_TC_all.json','r') as f:
#    TC_index = json.load(f)
#    f.close()

TC_index = load_features('./data/TC_index_preprocessed.json')

In [14]:
# Preprocess text for index. 

# remove english stop words. 
stop_words = list(stopwords.words('english'))
def rmstopwords(word):
    if word not in stop_words:
            return word
    else:
        return 'NaN'

# Remove terms that don't have letters (alphabetic letters)
def numbers(word):
    if word.isalpha():
        return word
    else:
        return 'NaN'

# check for characters that are not ascii.
def isEnglish(word):
    return word if word.isascii() else 'NaN'


def preprocess(doc):
    """Preprocesses text to prepare it for feature extraction. 
    
    Args:
        doc: String comprising the unprocessed contents. 
    
    Returns:
        String comprising the corresponding preprocessed text.
    """
    # Set all text to lower text
    doc = doc.lower()
    # Punctuations to be removed
    punctuation = '!"#$%&\'()*+,-./:;<=>?[\\]^_`{|}~'
    # Split by white spaces
    textlist = re.split(r'[\s*\n]',doc)
    # Remove punctuation of each word, make it to pandas series to easily clean up the empty elements
    words = pd.Series([rmstopwords(isEnglish(numbers(text.translate(str.maketrans('', '', punctuation))))) for text in textlist])
    # Removing empty elements
    words = words[words != 'NaN']
    text = ' '.join([word for word in words.to_list()])
    return text

def preprocess_multiple(docs):
    """Preprocesses multiple texts to prepare them for feature extraction. 
    
    Args:
        docs: List of strings, each consisting of the unprocessed contents of some email file. 
    
    Returns:
        List of strings, each comprising the corresponding preprocessed text.
    """
    # YOUR CODE HERE
    alldocs = []
    # Process each string for a document using preprocess() and append in alldocs
    for doc in docs:
        alldocs.append(preprocess(doc))
        
    # Returns the list of lists for all documents
    return alldocs


def split_capital(doc):
    labels = re.findall('([A-Z][a-z]*)', doc)
    return ' '.join(labels)

def split_type_labels(json_text):
    type_labels = {}
    for key in json_text.keys():
        type_labels[key] = split_capital(key[4:]).lower()
    return type_labels

def preprocess_index_text(index_json):
    num_docs = len(index_json) // 10
    for i, (type_id,body) in enumerate(index_json.items()):
        body['body'] = preprocess(body['body'])
        if i % num_docs == 0:
            print('{}% done'.format((i // num_docs)*10))
    save_features(TC_index,'./data/TC_index_preprocessed.json')
    return

In [22]:
# load all possible types from hierarchy 
type_family_features = load_features('./data/type_hierarchy_features.json')


In [23]:
# add missing types on the index with empty body field
for q in type_family_features.keys():
    if q not in TC_index.keys():
        TC_index[q] = {'body': ''}


In [81]:
# Count term frequency (TF) from a string. returns a Counter() object
def get_TF(text):
    count = cl.Counter(text.split())
    return count

# Get all the term frequencies for the index. 
def get_all_TF(index_doc):
    features = {}
    num_docs = len(index_doc)
    missing_types = []
# Calculates TF for the types with body. Save missing type ids on a list
    for i,(type_id,type_body) in enumerate(index_doc.items()):
        if len(type_body['body']) > 1:
            features[type_id] = get_TF(type_body['body'])
        else:
            missing_types.append(type_id)

# Agregates the TF information from childrens/siblings on types missing body
# Use childrens bodies or siblings if type doesn't have children
    for type_id in missing_types:
        x = cl.Counter()
        if len(type_family_features[type_id]['children']) > 0:
            for t_id in type_family_features[q]['children']:
                if t_id in features.keys():
                    x += features[t_id]
        if len(x.values()) > 1:
            features[type_id] = x
        else:
            for t_id in type_family_features[q]['siblings']:
                if t_id in features.keys():
                    x += features[t_id]
        if len(x) > 0:
            features[type_id] = x    
    return features


# Check if a term is in a document
# doc_terms - Counter() object with {term: frequency} for the document
def term_in_doc(doc_terms,term):
    return True if doc_terms[term] > 0 else False

# Count documents with a given term
#docs - index of documents - list of dict objects on the form 
#       {document_id : Counter() object with term frequencies for teh document }
# term - term/word that the idf is calculated
def count_docs_with_term(docs,term):
    counts = 0
    docs_term = []
    for doc in docs.keys():
        if term_in_doc(docs[doc],term):
            counts += 1
            docs_term.append(doc)
    return counts, docs_term

# Calculates the idf for all the terms in the list query_terms over all the documents 
# in the index_doc
# query_terms - list of words
# index_doc - index of documents - list of dict objects on the form 
#       {document_id : Counter() object with term frequencies for teh document }
def get_query_type_idf(query_terms,index_doc):
    idf = {}
    N = len(index_doc)
    for i, term in enumerate(query_terms):
                  
        if term not in idf.keys(): 
            docs_t_count,docs_t_ids = count_docs_with_term(index_doc,term)
            if docs_t_count != 0:
                idf[term] = math.log(N/docs_t_count)
            else:
                idf[term] = math.log(N)
   
           
    return idf 


# Calculates the idfs for a list of queries/type labels and return the features
# query_length, query_sum_idf, query_max_idf, query_avg_idf for each element on
# the list
# terms - list of queries/labels - list of a list of strings
# query_data - dict in the form {id: [terms]}
# index_doc - index of documents - list of dict objects on the form 
#       {document_id : Counter() object with term frequencies for teh document }
# returns a dict in the form {'query_length' : int, 'query_sum_idf': float, ...}

def extract_idf_features(terms,query_data,index_doc):
    # save and loads idfs for all terms on file
    try:
        idf = load_features('query_all_type_all_terms_idf_long.json')
        if idf == None:
            index_TFs = get_all_TF(index_doc) 
            idf = get_query_type_idf(query_data['uniques'],index_TFs)
            save_features(idf,'query_all_type_all_terms_idf_long.json')
    except:
        index_TFs = get_all_TF(index_doc) 
        idf = get_query_type_idf(query_data['uniques'],index_TFs)
        save_features(idf,'query_all_type_all_terms_idf_long.json')
    
    # calculates for query/label terms
    if len(terms) > 0:

        q_features = {}
        q_features['query_length'] = len(terms) #sum(get_TF(' '.join(terms)).values())
        q_idf = []
        for term in terms:        
            q_idf += [idf[term]]
        q_features['query_sum_idf'] = sum(q_idf)

        q_features['query_max_idf'] = max(q_idf)
        q_features['query_avg_idf'] = q_features['query_sum_idf']/len(terms)
    else:
        q_features['query_sum_idf'] = 0

        q_features['query_max_idf'] = 0
        q_features['query_avg_idf'] = 0
        q_features['query_length'] = 0
    
    return q_features

           

In [70]:
def extract_all_idf_features(query_data, index_doc, filename = False):
    """Extract all idf features for a list of queries/labels.
    
        Arguments:
            query_data: dict of queries/types with {id : string of terms (question/label)}.
            index_doc - index of documents - list of dict objects on the form 
            {document_id : Counter() object with term frequencies for teh document }
            filename: if given, load/saves the features from/in that filename
            
        Returns:
            X: List of feature vectors extracted for each pair of query and retrieved or relevant document. 
            y: List of corresponding labels.
    """
    X = []

    num_processed = 0
    N = len(query_data)
    try:
        features = load_features(filename)
        if features == None:
            features = {}
            filename = "query_idf_features"
    except:
        features = {}
        filename = "query_idf_features"

    for query_id,query_text in query_data.items():
        if query_id == 'uniques':
            continue
        if query_id not in features.keys():
            features[query_id] = {}

            query_terms = query_text.split()

            if len(query_terms) == 0:
                print('WARNING: query {} is empty after analysis; ignoring'.format(query_id))
                print(query_text)
                continue
            X.append(extract_idf_features(query_terms,query_data, index_doc))
            features[query_id]['X'] = X[-1]

        else:
            continue
                #print('Query not resource {}'.format(query['category']))

        num_processed += 1
        if num_processed % 100 == 0:
            save_features(features, filename)
            print('{} of {} queries processed...'.format(len(features.keys()),N))
    
    save_features(features, filename)
    print('All {} queries processed.'.format(N))
    return features

# Calculate the length of each type/doc and return a dict with 
# format {id: int (length)}
# index_doc - index of documents - list of dict objects on the form 
#       {document_id : Counter() object with term frequencies for teh document }
def extract_doc_length_features(index_doc):
    features = {}
# try to load the index term frequencies from file. If not possible, calculates index
# TF and save to file
    try:
        index_TFs = load_features('TC_index_TFs.json')
        if index_TFs == None:
            index_TFs = get_all_TF(index_doc) 
            save_features(index_TFs,'TC_index_TFs.json')
    except:
        index_TFs = get_all_TF(index_doc) 
        save_features(index_TFs,'TC_index_TFs.json')
   
   
    for i in index_doc.keys():
        features[i] = sum(index_TFs[i].values()) 
    return features

In [82]:
# Extract length of type centric index, also save to file
type_length_features = extract_doc_length_features(TC_index)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [84]:
index_TFs = load_features('TC_index_TFs.json')

In [90]:
index_TFs['dbo:Sound']

{'rapport': 2,
 'english': 1,
 'financial': 1,
 'report': 1,
 'monthly': 1,
 'business': 1,
 'magazine': 5,
 'published': 2,
 'oslo': 2,
 'norway': 1,
 'founded': 1,
 'owned': 1,
 'fora': 1,
 'medier': 1,
 'ivar': 1,
 'hippe': 1,
 'editor': 2,
 'chief': 2,
 'current': 1,
 'terje': 1,
 'aurdal': 1,
 'circulation': 1,
 'copies': 2,
 'angostura': 3,
 'diversion': 3,
 'dam': 4,
 'rio': 1,
 'grande': 1,
 'sandoval': 1,
 'county': 1,
 'new': 1,
 'mexico': 1,
 'near': 1,
 'algodones': 1,
 'north': 1,
 'bernalillo': 1,
 'diverts': 1,
 'water': 1,
 'main': 1,
 'irrigation': 1,
 'canal': 1,
 'serving': 1,
 'albuquerque': 1,
 'division': 1}

In [91]:
save_features(type_length_features,'type_legth_features.json')

In [36]:
# split labels for types ans set in lower case
type_labels = split_type_labels(TC_index)
type_labels

{'dbo:Disease': 'disease',
 'dbo:PopulatedPlace': 'populated place',
 'dbo:Region': 'region',
 'dbo:Settlement': 'settlement',
 'dbo:AdministrativeRegion': 'administrative region',
 'dbo:Place': 'place',
 'dbo:Agent': 'agent',
 'dbo:Person': 'person',
 'dbo:OfficeHolder': 'office holder',
 'dbo:TimePeriod': 'time period',
 'dbo:Book': 'book',
 'dbo:Work': 'work',
 'dbo:WrittenWork': 'written work',
 'dbo:Award': 'award',
 'dbo:TelevisionShow': 'television show',
 'dbo:Film': 'film',
 'dbo:PersonFunction': 'person function',
 'dbo:Scientist': 'scientist',
 'dbo:Philosopher': 'philosopher',
 'dbo:Writer': 'writer',
 'dbo:Country': 'country',
 'dbo:Athlete': 'athlete',
 'dbo:TennisPlayer': 'tennis player',
 'dbo:MusicalWork': 'musical work',
 'dbo:Song': 'song',
 'dbo:Sound': 'sound',
 'dbo:Document': 'document',
 'dbo:Organisation': 'organisation',
 'dbo:Satellite': 'satellite',
 'dbo:ArtificialSatellite': 'artificial satellite',
 'dbo:CelestialBody': 'celestial body',
 'dbo:Continent': 

In [37]:

# Find all unique terms in labels list
uniques = []
for label_id, terms in type_labels.items():
    for term in terms.split():
        if term not in uniques:
            uniques.append(term)
type_labels['uniques'] = uniques

In [59]:
type_labels

{'dbo:Disease': 'disease',
 'dbo:PopulatedPlace': 'populated place',
 'dbo:Region': 'region',
 'dbo:Settlement': 'settlement',
 'dbo:AdministrativeRegion': 'administrative region',
 'dbo:Place': 'place',
 'dbo:Agent': 'agent',
 'dbo:Person': 'person',
 'dbo:OfficeHolder': 'office holder',
 'dbo:TimePeriod': 'time period',
 'dbo:Book': 'book',
 'dbo:Work': 'work',
 'dbo:WrittenWork': 'written work',
 'dbo:Award': 'award',
 'dbo:TelevisionShow': 'television show',
 'dbo:Film': 'film',
 'dbo:PersonFunction': 'person function',
 'dbo:Scientist': 'scientist',
 'dbo:Philosopher': 'philosopher',
 'dbo:Writer': 'writer',
 'dbo:Country': 'country',
 'dbo:Athlete': 'athlete',
 'dbo:TennisPlayer': 'tennis player',
 'dbo:MusicalWork': 'musical work',
 'dbo:Song': 'song',
 'dbo:Sound': 'sound',
 'dbo:Document': 'document',
 'dbo:Organisation': 'organisation',
 'dbo:Satellite': 'satellite',
 'dbo:ArtificialSatellite': 'artificial satellite',
 'dbo:CelestialBody': 'celestial body',
 'dbo:Continent': 

In [51]:
# extract tc_label_idf_features: labels are the type name
type_label_features = extract_all_idf_features(type_labels,TC_index, filename = 'all_type_label_idf_features')

100 of 761 queries processed...
200 of 761 queries processed...
300 of 761 queries processed...
400 of 761 queries processed...
500 of 761 queries processed...
600 of 761 queries processed...
700 of 761 queries processed...
All 761 queries processed.


In [52]:
type_label_features

{'dbo:Disease': {'X': {'query_length': 1,
   'query_sum_idf': 0.387211667798814,
   'query_max_idf': 0.387211667798814,
   'query_avg_idf': 0.387211667798814}},
 'dbo:PopulatedPlace': {'X': {'query_length': 2,
   'query_sum_idf': 1.6873994539038122,
   'query_max_idf': 1.6094379124341003,
   'query_avg_idf': 0.8436997269519061}},
 'dbo:Region': {'X': {'query_length': 1,
   'query_sum_idf': 0.10975612713086441,
   'query_max_idf': 0.10975612713086441,
   'query_avg_idf': 0.10975612713086441}},
 'dbo:Settlement': {'X': {'query_length': 1,
   'query_sum_idf': 0.3969488430766723,
   'query_max_idf': 0.3969488430766723,
   'query_avg_idf': 0.3969488430766723}},
 'dbo:AdministrativeRegion': {'X': {'query_length': 2,
   'query_sum_idf': 0.495031685902812,
   'query_max_idf': 0.3852755587719476,
   'query_avg_idf': 0.247515842951406}},
 'dbo:Place': {'X': {'query_length': 1,
   'query_sum_idf': 0.07796154146971192,
   'query_max_idf': 0.07796154146971192,
   'query_avg_idf': 0.0779615414697119

In [76]:
save_features(type_label_features,'type_label_idf_features.json')

In [53]:
# format -> list of dicts
train = load_features('/home/daiana/Github/text-mining/project/DAT640_SMART/data/train_set_fixed.json')
validation = load_features('/home/daiana/Github/text-mining/project/DAT640_SMART/data/validation_set_fixed.json')
test = load_features('/home/daiana/Github/text-mining/project/DAT640_SMART/data/test_set_fixed.json')

In [60]:
# finding query terms for each query and unique terms from all queries.
# query is processed prior spliting terms. remove stop words and punctuation. 
# check if terms are ascii (limit to english alphabet) and set to lower case.
# return {query_ids: [terms]... 'uniques':[all unique terms]}
def find_all_query_terms(queries):
    query_terms = {}
    all_terms = []
    for query in queries:
        terms = preprocess(query['question'])
        query_terms[query['id']] = terms
        
        for term in terms.split():
            if term not in all_terms:
                all_terms.append(term)
    query_terms['uniques'] = all_terms
    save_features(query_terms,'query_terms')
    return query_terms    
        

In [61]:
all_queries = train + validation + test 

In [62]:
# finding query terms for each query and unique terms from all queries.
# Also preprocess question on query
query_terms = find_all_query_terms(all_queries)

In [63]:
query_terms

{'dbpedia_17780': 'science museums science museum people id',
 'dbpedia_21864': 'engagements alpine brigade taurinense',
 'dbpedia_7352': 'first name john lothrop motley equivalent gianni',
 'dbpedia_3353': 'electrical resistivity grivory equal',
 'dbpedia_410': 'award john williams nominated close encounters third kind',
 'dbpedia_23109': 'short shortest active nba player',
 'dbpedia_18488': 'allegiance albert kwesi ocran',
 'dbpedia_16842': 'size human genome',
 'dbpedia_5197': 'tianjin begin twinned administrative body hakodate',
 'dbpedia_6452': 'award keanu reeves receive also sandra bullock',
 'dbpedia_10995': 'managerdirector commands erich von manstein',
 'dbpedia_17264': 'work poul anderson receive prometheus award hall fame',
 'dbpedia_5852': 'many producer gladiator',
 'dbpedia_1669': 'garden earthly delights depict iconclass notation',
 'dbpedia_4294': 'rabies medical treatment health specialty',
 'dbpedia_8612': 'patientplus id death',
 'dbpedia_15195': 'city twin city osl

In [72]:
# Extract idf features for query-type
type_query_idf_features = extract_all_idf_features(query_terms,TC_index, filename = 'all_type_query_idf_features')

100 of 17255 queries processed...
200 of 17255 queries processed...
300 of 17255 queries processed...
400 of 17255 queries processed...
500 of 17255 queries processed...

601 of 17255 queries processed...
701 of 17255 queries processed...
801 of 17255 queries processed...
901 of 17255 queries processed...
1001 of 17255 queries processed...
1101 of 17255 queries processed...
1201 of 17255 queries processed...
1301 of 17255 queries processed...
1401 of 17255 queries processed...
1501 of 17255 queries processed...
1601 of 17255 queries processed...
1701 of 17255 queries processed...
1801 of 17255 queries processed...
1901 of 17255 queries processed...
2001 of 17255 queries processed...
2101 of 17255 queries processed...
2201 of 17255 queries processed...
2301 of 17255 queries processed...
2401 of 17255 queries processed...
2501 of 17255 queries processed...

2602 of 17255 queries processed...
2702 of 17255 queries processed...
2802 of 17255 queries processed...

2903 of 17255 queries proc

In [73]:
type_query_idf_features

{'dbpedia_17780': {'X': {'query_length': 6,
   'query_sum_idf': 2.9703263274242016,
   'query_max_idf': 1.7133375074522519,
   'query_avg_idf': 0.4950543879040336}},
 'dbpedia_21864': {'X': {'query_length': 4,
   'query_sum_idf': 8.872766176762706,
   'query_max_idf': 4.6874082842250635,
   'query_avg_idf': 2.2181915441906765}},
 'dbpedia_7352': {'X': {'query_length': 7,
   'query_sum_idf': 7.461786645022588,
   'query_max_idf': 2.804677036791282,
   'query_avg_idf': 1.0659695207175126}},
 'dbpedia_3353': {'X': {'query_length': 4,
   'query_sum_idf': 14.99585079655089,
   'query_max_idf': 6.633318433280377,
   'query_avg_idf': 3.7489626991377225}},
 'dbpedia_410': {'X': {'query_length': 8,
   'query_sum_idf': 3.917226423273633,
   'query_max_idf': 1.8050046959780757,
   'query_avg_idf': 0.4896533029092041}},
 'dbpedia_23109': {'X': {'query_length': 5,
   'query_sum_idf': 3.556977516356103,
   'query_max_idf': 1.9059306145680361,
   'query_avg_idf': 0.7113955032712206}},
 'dbpedia_18488

In [75]:
save_features(type_query_idf_features,'type_query_idf_features.json')